In [10]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SQLContext, Row
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from pyspark.ml import Pipeline
#from py4j.java_gateway import java_import
import sparknlp
from pyspark.sql.functions import udf
#sc.stop()

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)


/home/yogender/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning
/home/yogender/.local/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [3]:
rdd_e=sc.wholeTextFiles("./*.html")


In [65]:
rdd_e_split = rdd_e.map(lambda x: x[1])

In [132]:
import pandas as pd
from bs4 import BeautifulSoup
def processRecord_e(file):
    
    #import lxml.html.usedoctest
    #page = urlopen(url)
    #root = lxml.html.fromstring(tag_soup)
    soup = BeautifulSoup(file, "html.parser")
    #body = fromstring(tag_soup).find('.//body')
    #rtnVal = soup.prettify()
    #print (list(soup.children))
    classes = []

    for element in soup.find_all('ul', class_='params1'):
        for il in element.find_all('li'):
            text = il.get_text(strip=True).replace(u'\xa0', u' ')
            #text2 = il2.get_text(strip=True).replace(u'\xa0', u' ')
            #print ('text1 is :',text1)
            #print ('text2 is :',text2)
            classes.append(text.split(':')[3])
            #classes2.append(text2)
    #for element in classes():
    #    print (element)
    #df = pd.DataFrame([classes1, classes1])
    #print ('length of classes is:', classes)
    return classes


In [133]:
#processRecord_e(rdd_df)

In [134]:
convertUDF = lambda z: processRecord(z)

In [135]:
from pyspark.sql import Row

row = Row("val") # Or some other column name
rdd_df = rdd_e_split.map(row).toDF()

In [136]:
text_df_cleaned=rdd_df.withColumn('cleaned', udf(convertUDF, StringType())('val'))

In [137]:
text_df_cleaned

DataFrame[val: string, cleaned: string]

In [138]:
text_df_cleaned_flat = text_df_cleaned.select("cleaned").rdd.flatMap(lambda x: x)

In [139]:
from pyspark.sql import Row

row = Row("cleanedParams1") 

text_df_cleaned_flat= text_df_cleaned_flat.map(row).toDF()

In [140]:
pd.set_option('display.max_colwidth', None)

In [141]:
text_df_cleaned_flat_pd = text_df_cleaned_flat.select('cleanedParams1').toPandas()

In [142]:
text_df_cleaned_flat_pd 

,cleanedParams1
0,"[Celková cena:21 500 000 Kč za nemovitost, včetně provize, ID zakázky:32863, Aktualizace:16.02.2021, Stavba:Cihlová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní]"
1,"[Celková cena:5 200 000 Kč za nemovitost, včetně provize, ID zakázky:729030, Aktualizace:08.02.2021, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:2. podlaží z celkem 3, Užitná plocha:78m2, Plocha podlahová:78m2]"
2,"[Celková cena:10 854 900 Kč (420 000 EUR) za nemovitost, včetně provize, Aktualizace:26.01.2021, ID:261724, Stavba:Smíšená, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Podlaží:2. podlaží, Užitná plocha:132m2, Balkón:, Parkování:]"
3,"[Celková cena:5 565 000 Kč za nemovitost, včetně provize, Poznámka k ceně:+ poplatky 3.500 ,- Kč, ID zakázky:N03409, Aktualizace:18.02.2021, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:4. podlaží z celkem 6, Užitná plocha:55m2, Plocha podlahová:55m2]"
4,"[Celková cena:1 475 000 Kč za nemovitost, včetně provize, Poznámka k ceně:včetně právního servisu a provize RK, ID zakázky:0253-NP01146, Aktualizace:11.02.2021, Anuita:4 Kč, Stavba:Cihlová, Stav objektu:Novostavba, Vlastnictví:Družstevní, Převod do OV:Ne, Umístění objektu:Centrum obce, Podlaží:4. podlaží z celkem 6, Užitná plocha:57m2, Plocha podlahová:57m2]"
5,"[Celková cena:5 500 000 Kč za nemovitost, Poznámka k ceně:cena je včetně provize a služeb RK, ID zakázky:NPDO4772, Aktualizace:15.02.2021, Stavba:Cihlová, Stav objektu:Novostavba, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:1. podlaží z celkem 2, Užitná plocha:66m2]"
6,"[Celková cena:2 530 000 Kč za nemovitost, + provize RK, včetně právního servisu (k jednání), Poznámka k ceně:, 3500, ID zakázky:1579/1830, Aktualizace:15.02.2021, Stavba:Panelová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní, Podlaží:1. podlaží z celkem 5, Užitná plocha:44m2, Plocha podlahová:44m2]"
7,"[Celková cena:7 511 000 Kč za nemovitost, + provize RK, včetně právního servisu, Poznámka k ceně:+ provize RK, ID zakázky:202009Graf, Aktualizace:14.02.2021, Stavba:Cihlová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní, Umístění objektu:Centrum obce, Podlaží:4. podlaží z celkem 5 včetně 1 podzemního, Užitná plocha:58m2, Datum nastěhování:Ihned, Rok kolaudace:2021]"
8,"[Celková cena:1 850 000 Kč za nemovitost, Náklady na bydlení:3000,-Kč, Aktualizace:04.02.2021, ID:695644, Stavba:Panelová, Stav objektu:Před rekonstrukcí, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:10. podlaží, Užitná plocha:33m2]"
9,"[Celková cena:7 490 000 Kč za nemovitost, včetně provize, ID zakázky:N05057, Aktualizace:26.10.2020, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Podlaží:3. podlaží z celkem 4 včetně 1 podzemního, Užitná plocha:39m2]"
